In [1]:
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame

In [2]:
session = requests.Session()
session.headers.update({
    "Referer": "",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
})

In [3]:
clist = [1011,1023,1030,1041,1046,1052,1054,1057,1062,1064,1070,1073,1076]

In [ ]:
학과목록_리스트 = []

for c in clist:
    url = "https://www.sookmyung.ac.kr/sookmyungkr/%d/subview.do" % c

    r = session.get(url)
    if r.status_code != 200:
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
        raise Exception(msg)
    r.encoding = "utf-8"
    soup = BeautifulSoup(r.text)


    college_list = soup.select(".college_list")
    for item in college_list:
        h5El = item.select("h5")
        name = h5El[0].text.strip()

        viewEl = item.select(".view")
        view = viewEl[0].attrs["href"]
        if view.find("http://www.sookmyung.ac.kr") == -1:
            view = "http://www.sookmyung.ac.kr" + view

        pdfEl = item.select(".info") 
        pdf = pdfEl[0].attrs["href"] 
        
        homepageEl = item.select(".homepage")
        homepage = homepageEl[0].attrs['href']
        
        college_dict = {"학과이름": name, "상세페이지": view, "홈페이지": homepage}

        
        r = session.get(view)
        if r.status_code != 200:
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            print(msg)
            continue

        r.encoding = "utf-8"
        detailSoup = BeautifulSoup(r.text)

        infoEl = detailSoup.select(".college_info_data dl")
    
        for info in infoEl:
            dt = info.select("dt")[0].text.replace(":", "").strip() 
            dd = info.select("dd")[0].text.replace(":", "").strip()
        
            college_dict[dt] = dd
        
        학과목록_리스트.append(college_dict)
        
        if pdf.find(".pdf") > -1:
            pass
            r = session.get(pdf, stream=True)
            if r.status_code == 200:
                r.encoding = "utf-8"
                with open("%s.pdf" % name , 'wb') as f:
                    f.write(r.raw.read())

df = DataFrame(학과목록_리스트)
df.to_excel("result.xlsx")
df